In [1]:
!pip install fastai==2.7.9
!pip install fastkaggle==0.0.7
!pip install fasterai==0.1.11
!pip install ipython-autotime==0.3.1
!pip install torch==1.12.1
%load_ext autotime


[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


^C



[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


time: 0 ns (started: 2023-07-15 19:56:52 +07:00)



[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from fastkaggle import *
from fastai.vision.all import *
from fasterai.sparse.all import *
from torchvision.models import vgg16_bn

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


time: 6.02 s (started: 2023-05-28 05:45:10 +00:00)


In [ ]:
comp = 'paddy-diseases/paddy-diseases'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

time: 13.8 s (started: 2023-05-28 05:45:16 +00:00)


In [ ]:
dls = ImageDataLoaders.from_folder(path/"train", 
                                   valid_pct=0.25, 
                                   item_tfms=Resize(256), 
                                   batch_tfms=aug_transforms(   size=64, 
                                                                min_scale=0.75,
                                                                max_rotate=10., 
                                                                min_zoom=0.8, 
                                                                max_zoom=1.2, 
                                                            
                                                            ), 
                                   bs=32)

time: 15.4 s (started: 2023-05-28 05:45:30 +00:00)


In [ ]:
def run_once(func):
    """
    A decorator that runs a function only once and returns the result for all subsequent calls.
    """
    memo = {}
    def wrapper(*args, **kwargs):
        if func not in memo:
            memo[func] = func(*args, **kwargs)
        return memo[func]
    return wrapper


time: 656 µs (started: 2023-05-28 05:45:46 +00:00)


In [ ]:
@run_once
def get_model_file_size(model): 
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    model_size = param_size 
    print('Model size: {} Byte'.format(model_size))


time: 565 µs (started: 2023-05-28 05:45:46 +00:00)


In [ ]:
class MemoryCallback(Callback):
    def before_fit(self):
        get_model_file_size(self.learn.model)

time: 531 µs (started: 2023-05-28 05:45:46 +00:00)


In [ ]:
sp_cb=[
      # GradientAccumulation(32), 
      # SparsifyCallback(sparsity=12.5, 
      #                    granularity='filter', 
      #                    context='local', 
      #                    criteria=large_final, 
      #                    schedule=one_cycle
      #                    ),
      MemoryCallback(),
]

time: 686 µs (started: 2023-05-28 05:45:46 +00:00)


In [ ]:
model = vgg16_bn(num_classes=10)

time: 2.07 s (started: 2023-05-28 05:45:46 +00:00)


In [ ]:
learn = Learner(dls, 
                model, 
                metrics=[accuracy,
                         error_rate,
                        ]).to_fp16()
learn.unfreeze()
learn.recorder.train_metrics = True

time: 7.82 ms (started: 2023-05-28 05:45:49 +00:00)


In [ ]:
learn.fit_one_cycle(1, 3e-5, cbs=sp_cb)


epoch,train_loss,train_accuracy,train_error_rate,valid_loss,valid_accuracy,valid_error_rate,time
0,2.174146,0.210897,0.789103,1.953595,0.310577,0.689423,01:27


Model size: 537239848 Byte
time: 1min 27s (started: 2023-05-28 05:45:49 +00:00)
